<a href="https://colab.research.google.com/github/OseiasBeu/Cardiovascular-diseases-risk-project/blob/main/Pipeline_Cardiovascular_diseases_risk_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import requests
import zipfile
import io
import os
import numpy as np
import pickle

In [49]:
import requests
import os

github_url = 'https://api.github.com/repos/OseiasBeu/Cardiovascular-diseases-risk-project/contents/dataset.zip'

response = requests.get(github_url)


if response.status_code == 200:
    file_info = response.json()

    download_url = file_info.get('download_url')

    if download_url:
        response = requests.get(download_url)
        if response.status_code == 200:
            file_path = 'dataset.zip'

            with open(file_path, 'wb') as file:
                file.write(response.content)

            print(f'O arquivo foi baixado com sucesso e salvo em: {file_path}')
        else:
            print(f'Erro ao fazer o download do arquivo. Status Code: {response.status_code}')
    else:
        print('Não foi possível obter o URL de download do arquivo.')
else:
    print(f'Erro ao acessar a URL do GitHub. Status Code: {response.status_code}')


Erro ao fazer o download do arquivo. Status Code: 503


In [50]:
import zipfile

# Especifique o caminho para o arquivo ZIP baixado
zip_file_path = 'dataset.zip'

# Especifique o diretório de destino para a extração
extracted_dir_path = 'dataset/'

# Verifique se o arquivo ZIP existe
if not zipfile.is_zipfile(zip_file_path):
    print(f'O arquivo {zip_file_path} não é um arquivo ZIP válido.')
else:
    # Crie o diretório de destino se ele não existir
    if not os.path.exists(extracted_dir_path):
        os.makedirs(extracted_dir_path)

    # Abra o arquivo ZIP para extração
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extraia todo o conteúdo para o diretório de destino
        zip_ref.extractall(extracted_dir_path)

    print(f'O arquivo ZIP foi extraído com sucesso para o diretório: {extracted_dir_path}')


O arquivo ZIP foi extraído com sucesso para o diretório: extracted_data/


In [51]:
import requests

# URL do arquivo que você deseja baixar
file_url = 'https://github.com/OseiasBeu/Cardiovascular-diseases-risk-project/raw/main/model/dtc_model_cdr_r.pkl'

# Nome do arquivo de destino
file_name = 'dtc_model_cdr_r.pkl'

# Faça o download do arquivo
response = requests.get(file_url)

# Verifique se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Salve o arquivo no disco
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'O arquivo foi baixado com sucesso e salvo como {file_name}')
else:
    print(f'Erro ao fazer o download do arquivo. Status Code: {response.status_code}')


O arquivo foi baixado com sucesso e salvo como dtc_model_cdr_r.pkl


In [52]:
data = pd.read_csv('/content/dataset/dataset/CVD_cleaned.csv')

In [53]:
data.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0


In [54]:
data.columns

Index(['General_Health', 'Checkup', 'Exercise', 'Heart_Disease', 'Skin_Cancer',
       'Other_Cancer', 'Depression', 'Diabetes', 'Arthritis', 'Sex',
       'Age_Category', 'Height_(cm)', 'Weight_(kg)', 'BMI', 'Smoking_History',
       'Alcohol_Consumption', 'Fruit_Consumption',
       'Green_Vegetables_Consumption', 'FriedPotato_Consumption'],
      dtype='object')

In [55]:
mock_data = {
    'General_Health': 'Poor',
    'Checkup': 'Within the past 2 years',
    'Exercise': 'No',
    'Skin_Cancer': 'No',
    'Other_Cancer': 'No',
    'Depression': 'No',
    'Diabetes': 'No',
    'Arthritis': 'Yes',
    'Sex': 'Female',
    'Age_Category': '70-74',
    'Height_(cm)': 150.0,
    'Weight_(kg)': 32.66,
    'BMI': 14.54,
    'Smoking_History': 'Yes',
    'Alcohol_Consumption': 0.0,
    'Fruit_Consumption': 30.0,
    'Green_Vegetables_Consumption': 16.0,
    'FriedPotato_Consumption': 12.0
    }

In [92]:
# Transforme o dicionário em um DataFrame
data = pd.DataFrame([mock_data])

# Agora, 'df' conterá os dados do dicionário como um DataFrame
data.head()

,General_Health,Checkup,Exercise,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0


In [95]:
mapper_hd = {
    'No':0,
    'Yes':1,
    np.nan: 1
}

mapper_sex = {
    'Female':0,
    'Male':1,
    np.nan: 1
}

In [96]:
# data['Heart_Disease'] = data['Heart_Disease'].map(mapper_hd,na_action=None)
data['Sex'] = data['Sex'].map(mapper_sex,na_action=None)
data['Exercise'] = data['Exercise'].map(mapper_hd,na_action=None)
data['Skin_Cancer'] = data['Skin_Cancer'].map(mapper_hd,na_action=None)
data['Other_Cancer'] = data['Other_Cancer'].map(mapper_hd,na_action=None)
data['Depression'] = data['Depression'].map(mapper_hd,na_action=None)
data['Arthritis'] = data['Arthritis'].map(mapper_hd,na_action=None)
data['Smoking_History'] = data['Smoking_History'].map(mapper_hd,na_action=None)

In [97]:
data.loc[data['Checkup'] == 'Within the past year', 'last_Checkup_years'] = '<1'
data.loc[data['Checkup'] == 'Within the past 2 years', 'last_Checkup_years'] = '<2'
data.loc[data['Checkup'] == 'Within the past 5 years', 'last_Checkup_years'] = '<5'
data.loc[data['Checkup'] == '5 or more years ago', 'last_Checkup_years'] = '>5'
data.loc[data['Checkup'] == 'Never', 'last_Checkup_years'] = 'Never'

In [98]:
data.loc[data['Diabetes'] == 'No', 'diabetes?'] = 'No'
data.loc[data['Diabetes'] == 'Yes', 'diabetes?'] = 'Yes'
data.loc[data['Diabetes'] == 'No, pre-diabetes or borderline diabetes', 'diabetes?'] = 'Pre_diabetes_boderline'
data.loc[data['Diabetes'] == 'Yes, but female told only during pregnancy', 'diabetes?'] = 'Yes_scovered_in_pregnancy'

In [100]:
# Get one hot encoding of columns
one_hot = pd.get_dummies(data[['General_Health','Age_Category','last_Checkup_years', 'diabetes?']])
# Drop columns as it is now encoded
data = data.drop(['General_Health', 'Age_Category','last_Checkup_years','diabetes?','Checkup','Diabetes'],axis = 1)
# Join the encoded df
df = data.join(one_hot)

In [101]:
columns_Age = [
            'Age_Category_18-24', 'Age_Category_25-29',
            'Age_Category_30-34', 'Age_Category_35-39', 'Age_Category_40-44',
            'Age_Category_45-49', 'Age_Category_50-54', 'Age_Category_55-59',
            'Age_Category_60-64', 'Age_Category_65-69', 'Age_Category_70-74',
            'Age_Category_75-79', 'Age_Category_80+'
            ]

In [102]:
columns_GeneralHealth = [
            'General_Health_Excellent', 'General_Health_Fair',
            'General_Health_Good', 'General_Health_Poor',
            'General_Health_Very Good'
          ]

In [103]:
columns_last_checkup = [
                        'last_Checkup_years_<1',
                        'last_Checkup_years_<2', 'last_Checkup_years_<5',
                        'last_Checkup_years_>5', 'last_Checkup_years_Never'
                        ]

In [104]:
columns_diabetes = [
                    'diabetes?_No',
                    'diabetes?_Pre_diabetes_boderline', 'diabetes?_Yes',
                    'diabetes?_Yes_scovered_in_pregnancy'
                    ]

In [105]:
def create_features_with_zero(columns):
  for column in columns:
    if column not in df.columns:
      df[column] = 0

In [106]:
create_features_with_zero(columns_Age)
create_features_with_zero(columns_last_checkup)
create_features_with_zero(columns_GeneralHealth)
create_features_with_zero(columns_diabetes)

In [118]:
pd.read_csv('/content/extracted_data/dataset/X_train_r_cdr.csv',sep=';').head()

,Unnamed: 0,Exercise,Skin_Cancer,Other_Cancer,Depression,Arthritis,Sex,Height_(cm),Weight_(kg),BMI,...,Age_Category_80+,last_Checkup_years_<1,last_Checkup_years_<2,last_Checkup_years_<5,last_Checkup_years_>5,last_Checkup_years_Never,diabetes?_No,diabetes?_Pre_diabetes_boderline,diabetes?_Yes,diabetes?_Yes_scovered_in_pregnancy
0,39183,1,0,0,0,0,1,173.0,72.57,24.33,...,0,0,0,1,0,0,1,0,0,0
1,242664,0,0,0,1,0,1,175.0,83.91,27.32,...,0,1,0,0,0,0,1,0,0,0
2,70046,1,0,1,0,0,1,168.0,74.84,26.63,...,0,1,0,0,0,0,1,0,0,0
3,361062,1,0,0,1,1,0,157.0,58.97,23.78,...,0,1,0,0,0,0,1,0,0,0
4,192037,1,0,0,0,0,1,175.0,65.32,21.26,...,0,1,0,0,0,0,1,0,0,0


In [115]:
df['Unnamed: 0'] =

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 41 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Exercise                             1 non-null      int64  
 1   Skin_Cancer                          1 non-null      int64  
 2   Other_Cancer                         1 non-null      int64  
 3   Depression                           1 non-null      int64  
 4   Arthritis                            1 non-null      int64  
 5   Sex                                  1 non-null      int64  
 6   Height_(cm)                          1 non-null      float64
 7   Weight_(kg)                          1 non-null      float64
 8   BMI                                  1 non-null      float64
 9   Smoking_History                      1 non-null      int64  
 10  Alcohol_Consumption                  1 non-null      float64
 11  Fruit_Consumption                   

In [109]:
with open('dtc_model_cdr_r.pkl', 'rb') as file:
    modelo_treinado = pickle.load(file)

In [110]:
print(type(modelo_treinado))

<class 'sklearn.tree._classes.DecisionTreeClassifier'>


In [111]:
previsoes = modelo_treinado.predict_proba(df)

ValueError: ignored